# Задача по анализу поиска в Яндекс Картинках

# Содержание 

* [1. Описание задачи](#0)
* [2. Установка инструментов](#1)
    * [2.1 Список пакетов Python](#1-1)
    * [2.2 Импорт пакетов](#1-2)
* [3. Загрузка данных](#2)
* [4. Предварительная обработка данных](#3)
* [5. Определение диапазона дат для анализа](#4)
* [6. Рассчет кол-ва запросов "ютуб"](#5)
* [7. Определение топ-10 самых частых запросов](#6)
* [8. Анализ трафика в течение дня](#7)
* [9. Определение контрастных тематик](#8)
* [10. Выводы](#9)

<a id='0'></a>
# Часть 1. Описание задачи

В данной работе будем проверять гипотезу, что интересы людей, пользующихся поиском по картинкам на мобильных устройствах и на компьютерах заметно отличаются.

Задачи, которые будем решать в рамках данной работы:
 * Определим даты диапазона для нашего анализа;
 * Рассчитаем кол-во запросов с текстом "ютуб" по каждой платформе;
 * Выведем топ-10 самых частых запросов по платформам;
 * Посмотрим, как отличается трафик запросов в течение дня;
 * Выделим тематики запросов, контрастные для платформ темы с разными долями.


<a id='1'></a>
# Часть 2. Установка инструментов

<a id='1-1'></a>
## 2.1. Перечень пакетов, которые мы будем использовать в данной работе
* pandas;
* plotly.express.

<a id='1-2'></a>
## 2.2. Импорт пакетов

In [152]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

<a id='2'></a>
# Часть 3. Загрузка данных

Наши данные будем брать из файла data.tsv, в котором содержится семпл запросов к Яндекс Картинкам за несколько недель.


In [ ]:
# Загрузим данные из файла
df = pd.read_csv('data.tsv', sep='\t', header=None) # указываем header=None, так как отсутствуют заголовки
df.head(10)

,0,1,2
0,малевич картины,1631806465,desktop
1,психология,1631781583,touch
2,с днём рождения лена,1631771563,touch
3,зверополис фильмы,1631787599,touch
4,алабай собака фото,1631786645,touch
5,бактериофаг,1631816202,desktop
6,полина гагарина,1631804005,touch
7,кадр из фильма,1631770837,desktop
8,погода,1631780741,touch
9,кухни дизайн,1631813821,touch


<a id='3'></a>
# Часть 4. Предварительная обработка данных

In [22]:
# Зададим имена столбцов и их порядок
df.columns = ['request', 'date', 'platform']
df = df.loc[:, ['date', 'request', 'platform']]
df

,date,request,platform
0,1631806465,малевич картины,desktop
1,1631781583,психология,touch
2,1631771563,с днём рождения лена,touch
3,1631787599,зверополис фильмы,touch
4,1631786645,алабай собака фото,touch
...,...,...,...
1114360,1631475653,бабочка,touch
1114361,1631462934,все республики россии,touch
1114362,1631432252,кухня в стиле лофт,touch
1114363,1631449347,топор,desktop


In [23]:
# Проверим данные на пропуски (NaN значения)
df.isnull().sum()

date        0
request     0
platform    0
dtype: int64

In [29]:
# Приведем дату из timestamp к формату "yyyy-mm-dd hh:mm:ss" и часовому поясу Москвы
df['date'] = pd.to_datetime(df['date'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Europe/Moscow')
df

,date,request,platform
805636,2021-09-01 00:00:00+03:00,погода,touch
722734,2021-09-01 00:00:01+03:00,3д Slumdog,desktop
1073088,2021-09-01 00:00:01+03:00,бравл старс игры,touch
421467,2021-09-01 00:00:03+03:00,нива тревел,desktop
886176,2021-09-01 00:00:06+03:00,школьница,touch
...,...,...,...
940448,2021-09-21 23:59:47+03:00,открытка с днём рождения женщине красивые,desktop
351399,2021-09-21 23:59:47+03:00,акацуки,touch
290310,2021-09-21 23:59:48+03:00,Катерина Ковальчук,desktop
1111998,2021-09-21 23:59:54+03:00,открытки с днем рождения мужчине,touch


In [30]:
# Отсортируем датафрейм по дате для более удобной работы
df = df.sort_values(by='date')
df

,date,request,platform
805636,2021-09-01 00:00:00+03:00,погода,touch
722734,2021-09-01 00:00:01+03:00,3д Slumdog,desktop
1073088,2021-09-01 00:00:01+03:00,бравл старс игры,touch
421467,2021-09-01 00:00:03+03:00,нива тревел,desktop
886176,2021-09-01 00:00:06+03:00,школьница,touch
...,...,...,...
351399,2021-09-21 23:59:47+03:00,акацуки,touch
940448,2021-09-21 23:59:47+03:00,открытка с днём рождения женщине красивые,desktop
290310,2021-09-21 23:59:48+03:00,Катерина Ковальчук,desktop
1111998,2021-09-21 23:59:54+03:00,открытки с днем рождения мужчине,touch


<a id='4'></a>
# Часть 5. Определение диапазона дат для анализа

Для определения диапазона дат возьмем минимальное и максимальное значение по дате.

In [32]:
min_date = df['date'].min()
max_date = df['date'].max()

print(f'Диапазон дат для анализа: {min_date} - {max_date}')

Диапазон дат для анализа: 2021-09-01 00:00:00+03:00 - 2021-09-21 23:59:59+03:00


Получаем, что диапазон дат для нашего анализа: 2021-09-01 00:00:00 - 2021-09-21 23:59:59.

<a id='5'></a>
# Часть 6. Расчет количества запросов "ютуб"

Рассчитаем количество запросов с текстом "ютуб" по каждой платформе. Для этого отфильтруем данные, сгруппирруем по платформе, посчитаем запросы и визуализируем результаты.

In [74]:
# Сперва отфильтруем данные по тексту "ютуб"
youtube_requests = df.query("request == 'ютуб'").groupby('platform').agg({'request': 'count'}).reset_index()
youtube_requests

,platform,request
0,desktop,714
1,touch,476


In [153]:
# Визуализируем результаты
fig = px.bar(
    youtube_requests,
    x='platform',
    y='request',
    color='platform',
    text='request',
    labels={
        'request': 'Количество запросов, шт',
        'platform': 'Платформа'
    }
)

fig.update_traces(
    textposition='outside'
)

fig.update_layout(
    title_text="Количество запросов 'ютуб' по платформам",
    title_x=0.5,
    title_font_size=16,
    showlegend=False,
)

fig.show()

По итогу мы видим, что пользователи с ПК и ноутбуков гораздо чаще ищут "ютуб" в запросах, нежели пользователи мобильных устройств. Разница ровно в 1.5 раза.

<a id='6'></a>
# Часть 7. Определение топ-10 самых частотных запросов

Определим топ-10 самых частых запросов по платформам. Для этого сперва сгруппируем наши данные по платформе и запросу и посчитаем количество запросов, затем выведем топ-10 запросов по каждой платформе и визуализируем результат.

In [34]:
# Сперва сгруппируем наши данные по платформе и запросу, посчитаем количество запросов и отсортируем 
top_queries = (
    df
    .groupby(['platform', 'request'])['request']
    .value_counts()
    .reset_index(name='query_cnt')
    .sort_values(['platform','query_cnt'], ascending=[True, False])
)
top_queries

,platform,request,query_cnt
9108,desktop,календарь 2021,2804
20808,desktop,таблица менделеева,2631
9482,desktop,картинки,1647
3359,desktop,английский алфавит,1293
14159,desktop,обои на рабочий стол,1143
...,...,...,...
55473,touch,японское образование,1
55475,touch,японское психология,1
55476,touch,японское путешествия,1
55477,touch,японское спорт,1


In [90]:
# Теперь выведем топ-10 запросов по каждой платформе
top10_per_platform = top_queries.groupby('platform').head(10).reset_index(drop=True)
top10_per_platform

,platform,request,query_cnt
0,desktop,календарь 2021,2804
1,desktop,таблица менделеева,2631
2,desktop,картинки,1647
3,desktop,английский алфавит,1293
4,desktop,обои на рабочий стол,1143
5,desktop,Одноклассники (социальная сеть),1116
6,desktop,таблица квадратов,877
7,desktop,алфавит,874
8,desktop,таблица умножения,867
9,desktop,карта мира,795


In [148]:
# Визуализируем результаты
fig = px.bar(
    top10_per_platform,
    y='request',  
    x='query_cnt',
    color='platform',
    orientation='h',  
    barmode='group',
    labels={
        'request': 'Поисковый запрос',
        'query_cnt': 'Количество запросов, шт',
        'platform': 'Платформа'
    }
)

fig.update_layout(
    title='Топ‑10 запросов по платформам',
    title_x=0.5,
    title_font_size=16,
    )

fig.show()


Глядя на график с топ-10 запросами по платформам, можно заметить, что количество запросов с мобильных платформ кратно выше запросов с ПК и ноутбуков. Можно сделать вывод о том, что основная часть пользователей Яндекс картинок - это преимущественно пользователи мобильных платформ. 

Также можно выделить характерные различия в самих запросах. Мы видим, что пользователи с ПК и ноутбуков в основном ищут информацию по учебе и работе, тогда как характер запросов пользователей мобильных платформ смещен в сторону быстрых событий (поздравлений с днем рождения, просмотра погоды, новостей).

<a id='7'></a>
# Часть 8. Анализ трафика запросов в течение дня 

Проанализируем общий трафик по запросам в течении дня, а также посмотрим более детально самые частые запросы каждого часа.

Для того чтобы произвести анализх трафика в течении дня, сперва переведем даты запросов в часы и далее сгруппируем наши запросы. Затем сделаем визуализацию нашего временного ряда.

In [ ]:
# Добавим новый столбец с часами
df['hour'] = df['date'].dt.hour

# Создадим датафрейм глобального траффика по платформам
traffic_global = (
    df.groupby(['platform', 'hour'], as_index=False)
    .agg({'request': 'count'})
)
traffic_global.head()

,platform,hour,request
0,desktop,0,6232
1,desktop,1,3612
2,desktop,2,2477
3,desktop,3,2234
4,desktop,4,2805


In [149]:
# Визуализируем полученный результат
fig = px.line(
    traffic_global,
    x='hour',
    y='request',
    facet_col='platform',
    color_discrete_sequence=px.colors.qualitative.Set2,
    labels={
        'hour': 'Час суток',
        'request': 'Количество запросов',
        'platform': 'Платформа'
    }
)

fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='LightGray'
)

fig.update_xaxes(
    dtick=2,  
    showgrid=True
)

fig.update_layout(
    title_text="Часовой трафик запросов по платформам",
    title_x=0.5,
    title_font_size=16,
    
    yaxis_title="Количество запросов",
    
    plot_bgcolor='white',
    height=500,
    width=1200
)

fig.show()


По нашим графиках хорошо видно различие в количестве трафика по платформам - преобладают пользователи Яндекс Картинок с мобильных устройств. Запросы пользователей ПК и ноутбуков плавно начинают расти с началом рабочего дня и так же плавно падают к его кончанию, пик запросов приходится на 16:00. 

С мобильными платформами ситуация совсем иная. Мы отчестливо видим бимодальное распределение трафика: обе моды совпадают со временем начала рабочего дня и его окончанием (первый пик трафика приходится на 10:00, второй пик трафика попадает на 20:00).

Теперь посмотрим на трафик самых частых запросов каждого часа. Возьмем топ-3 категории каждого часа.

In [ ]:
# Найдем топ-3 запроса по каждому часу
top_per_hour = (
    df
    .groupby(['hour', 'request'])
    .size()
    .to_frame('count')
    .sort_values(['hour', 'count'], ascending=[True, False])
    .groupby('hour')
    .head(3)
    .reset_index()
)

most_freq_request = set(top_per_hour['request'])
most_freq_request


{'айфон 13',
 'английский алфавит',
 'доброе утро',
 'здоровье',
 'игры',
 'календарь 2021',
 'картинки',
 'кулинария',
 'наука',
 'образование',
 'погода',
 'путешествия',
 'с днем рождения',
 'с днём рождения',
 'с днём рождения женщине',
 'с днём рождения мужчине',
 'таблица менделеева',
 'технологии',
 'фильмы'}

In [41]:
# Создадим датафрейм с топ запросами по часам
traffic_top_request = (
    df[df['request'].isin(most_freq_request)]
    .groupby(['platform', 'request', 'hour'], as_index=False)
    .size()
)
traffic_top_request

,platform,request,hour,size
0,desktop,айфон 13,0,8
1,desktop,айфон 13,1,1
2,desktop,айфон 13,2,3
3,desktop,айфон 13,3,2
4,desktop,айфон 13,5,2
...,...,...,...,...
902,touch,фильмы,19,122
903,touch,фильмы,20,155
904,touch,фильмы,21,166
905,touch,фильмы,22,164


In [150]:
# Визуализируем полученный датафрейм
fig = px.line(
    traffic_top_request,
    x='hour',
    y='size',
    color='request',
    facet_col='platform',
    labels={
        'hour': 'Час суток',
        'request': 'Запросы',
        'platform': 'Платформа',
        'size': 'Количество запросов, шт.'
    }
)

fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='LightGray'
)

fig.update_xaxes(
    dtick=2,  
    showgrid=True
)

fig.update_layout(
    title_text="Часовой трафик топ запросов по платформам",
    title_x=0.5,
    title_font_size=16,
    
    plot_bgcolor='white',
    height=600,
    width=1280
)

fig.show()

Из графика мы видим, что тематика топ запросов по платформам сильно отличается. На ПК и ноутбуках превалируют запросы по учебе и работе (превышают все остальные в 2 раза и более). На мобильных платформах люди чаще всего ищут поздравления с днем рождения, пожелания доброго утра. Эти категории так же превышают все остальные в 2 раза и более. 

Характерано, что пик запросов на мобильных платформах приходится именно на начало рабочего дня (8:00-9:00), тогда как на ПК и ноутбуках мы видим более равномерное распределение количества запросов с начала (8:00-9:00) и по конец рабочего дня (17:00-20:00). 

<a id='8'></a>
# Часть 9. Определение контрастных тематик

Найдем контрастные тематики запросов, доли которых отличаются между платформами.

Сперва определим доли каждого запроса внутри платформы, затем найдем выделим одинаковые запросы по двум платформам и посчитаем разницу долей запросов между платформами. В итоге, выведем топ-10 контрастных запросов с самой большой разницей долей.

In [ ]:
# Посчитаем количество запросов по платформам
cnt_paltform = (
    df
    .groupby(['platform', 'request'])
    .size()
    .reset_index(name='cnt')
)
cnt_paltform

,platform,request,cnt
0,desktop,+18,11
1,desktop,002,4
2,desktop,02 аниме,34
3,desktop,02 кино,1
4,desktop,02 кулинария,1
...,...,...,...
55503,touch,ёжики,14
55504,touch,幼女,14
55505,touch,视频,69
55506,touch,보지,13


In [44]:
# Посчитаем долю запроса внутри платформы
cnt_paltform['share'] = (
    cnt_paltform['cnt'] / cnt_paltform.groupby('platform')['cnt'].transform('sum') # используем transform чтобы избежать merge
)
cnt_paltform

,platform,request,cnt,share
0,desktop,+18,11,0.000028
1,desktop,002,4,0.000010
2,desktop,02 аниме,34,0.000088
3,desktop,02 кино,1,0.000003
4,desktop,02 кулинария,1,0.000003
...,...,...,...,...
55503,touch,ёжики,14,0.000019
55504,touch,幼女,14,0.000019
55505,touch,视频,69,0.000095
55506,touch,보지,13,0.000018


In [45]:
# Уберем редкие запросы (< 100)
good_requests = cnt_paltform.query('cnt > 100')
good_requests

,platform,request,cnt,share
11,desktop,1 сентября,277,0.000713
17,desktop,1 сентября картинки,223,0.000574
66,desktop,13 карт,282,0.000726
107,desktop,18+,143,0.000368
153,desktop,3 сентября,107,0.000276
...,...,...,...,...
55344,touch,я календарь переверну,111,0.000153
55347,touch,я календарь переверну и снова 3 сентября,107,0.000147
55405,touch,яндекс,928,0.001278
55411,touch,яндекс картинки,177,0.000244


In [46]:
# Оставим запросы, которые есть на обеих платформах
comm_requests = (
    good_requests
    .groupby('request')['platform']
    .nunique()
)
comm_requests = comm_requests[comm_requests == 2].index
# comm_requests
good_requests = good_requests[good_requests['request'].isin(comm_requests)]
good_requests

,platform,request,cnt,share
11,desktop,1 сентября,277,0.000713
17,desktop,1 сентября картинки,223,0.000574
66,desktop,13 карт,282,0.000726
107,desktop,18+,143,0.000368
153,desktop,3 сентября,107,0.000276
...,...,...,...,...
54742,touch,чёрный фон без ничего,290,0.000399
55236,touch,эстетика,106,0.000146
55336,touch,ютуб,476,0.000656
55405,touch,яндекс,928,0.001278


In [47]:
# Создадим сводную таблицу с общими запросами по платформам. Значеия отображают доли.
desk_touch = (
    good_requests
    .pivot(
        index='request',
        columns='platform',
        values='share'
    )
)
desk_touch

platform,desktop,touch
request,,
1 сентября,0.000713,0.001373
1 сентября картинки,0.000574,0.000899
13 карт,0.000726,0.000702
18+,0.000368,0.001136
3 сентября,0.000276,0.000665
...,...,...
чёрный фон без ничего,0.000337,0.000399
эстетика,0.000276,0.000146
ютуб,0.001839,0.000656


In [ ]:
# Посчитаем абсолютную разницу долей запросов между платформами, а так же разницу в процентах. 
desk_touch['diff'] = (desk_touch['desktop'] - desk_touch['touch']).abs() # Возьмем значение по модулю, чтобы уйти от минусовых значений
desk_touch['diff_percent'] = (desk_touch['desktop'] - desk_touch['touch']) * 100
desk_touch

platform,desktop,touch,diff,diff_percent
request,,,,
1 сентября,0.000713,0.001373,0.000660,-0.065966
1 сентября картинки,0.000574,0.000899,0.000325,-0.032497
13 карт,0.000726,0.000702,0.000024,0.002393
18+,0.000368,0.001136,0.000768,-0.076790
3 сентября,0.000276,0.000665,0.000390,-0.038961
...,...,...,...,...
чёрный фон без ничего,0.000337,0.000399,0.000062,-0.006199
эстетика,0.000276,0.000146,0.000130,0.012960
ютуб,0.001839,0.000656,0.001183,0.118340


In [ ]:
# Выберем самые контрастные запросы, доли которых сильно различаются. Возьмем топ-10 запросов.
result = desk_touch.sort_values('diff', ascending=False)
result = result.head(10)
result

platform,desktop,touch,diff,diff_percent
request,,,,
с днём рождения женщине,0.001290,0.006753,0.005462,-0.546214
с днём рождения,0.000734,0.005463,0.004729,-0.472939
календарь 2021,0.007222,0.003081,0.004141,0.414105
с днём рождения мужчине,0.000855,0.004990,0.004135,-0.413458
таблица менделеева,0.006776,0.002888,0.003888,0.388829
с днем рождения,0.001136,0.004139,0.003003,-0.300272
обои на рабочий стол,0.002944,0.000140,0.002803,0.280340
погода,0.001290,0.003911,0.002621,-0.262094
фильмы,0.001208,0.003782,0.002574,-0.257390


In [157]:
# Визуализируем результат
platform = ['desktop' if x > 0 else 'touch' for x in result['diff_percent']]

fig = px.bar(
    result,
    x='diff_percent',
    y=result.index,
    orientation='h',
    color=platform,
    text=result['diff_percent'].round(2).astype(str) + '%',
    labels={
        'diff_percent': 'Разница долей, %',
        'request': 'Поисковый запрос',
    },
)

fig.add_vline(
    x=0,
    line_width=1,
    line_dash='solid'
)

fig.update_traces(
    textposition='outside',
    hovertemplate=(
        '<b>Поисковый запрос:</b> %{y}<br>'
        '<b>Разница долей:</b> %{x:.2f}%<br>'
    )
)

fig.update_layout(
    title='Контраст долей топ-10 одинаковых запросов между платформами',
    title_x=0.5,
    title_font_size=16,
    legend_title_text='Платформы',
    bargap=0.15
)

fig.show()

Для визуализации контраста долей используется график расходящихся столбцов (diverging bar chart). Он позволяет наглядно показать направление и величину смещения между запросами для наших платформ. 

Значения правее нуля указывают на перекос в сторону ПК и ноутбуков, левее - в сторону мобильных устройств.

В итоге, мы сравниили долю топ-10 смежных запросов между двумя платформами.

Из графика мы видим, что процентное различие по топ-10 запросам находится в диапазоне от 0,55 до 0,26 %. Это все может говорить нам о том, что пользовательские сценарии на платформах различаются.

<a id='9'></a>
# Часть 10. Выводы

### Краткие итоги
В результате данной работы мы:
* Убедились, что количество запросов на двух платформах кратно отличается.
* Увидели, что тематика топ запросов отличается между платформами.
* Узнали, насколько сильно отличается динамика трафика запросов в течении дня по платформам.
* Посчитали разницу долей контрастных запросов между двумя платформами.

### Общие заключительные тезисы
В результате проделанной работы мы подтвердили гипотезу о том, что интересы людей, пользующихся поиском по картинкам на мобильных устройствах и на компьютерах заметно отличаются.

Ответ на главный вопрос работы мы получили, что свидетельствует об успехе. 
